In [1]:
from google.colab import drive
drive._mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [3]:
!pip install transformers
!pip install -U sentence-transformers

In [4]:
%cd /content/drive/MyDrive/20211/rec-sys/mind-recomendation/naml

/content/drive/MyDrive/20211/rec-sys/mind-recomendation/naml


In [5]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm 
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.naml import NAMLModel
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


## Prepare Parameters

In [6]:
epochs = 1
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'small'

## Download and load data

In [7]:
data_path = data_path = "/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small"

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding_all.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict_all.pkl")
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'naml.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

## Create hyper-parameters

In [8]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file, 
                          subvertDict_file=subvertDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 1, 'batch_size': 32, 'show_step': 100000, 'title_size': 384, 'body_size': 50, 'his_size': 50, 'vert_num': 17, 'subvert_num': 249, 'data_format': 'naml', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'naml', 'dense_activation': 'relu', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small/utils/embedding_all.npy', 'wordDict_file': '/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small/utils/word_dict_all.pkl', 'userDict_file': '/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small/utils/uid2index.pkl', '

In [9]:
# iterator = MINDAllIterator(hparams)  # mind all thì có thêm load cả body nữa 

In [10]:
print(train_news_file)
print(train_behaviors_file)

/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small/train/news.tsv
/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small/train/behaviors.tsv


In [11]:
# iterator.load_data_from_file(news_file= train_news_file,behavior_file= train_behaviors_file)

In [12]:
# import pickle
# with open(wordDict_file, "rb") as f:
#     a = pickle.load(f)

In [13]:
# len(a)  # lúc load cái từ lên nó đã cho qua một từ điển 60992 từ rồi vcl 

In [14]:
iterator = MINDAllIterator

## Train the NAML model

In [15]:
model = NAMLModel(hparams, iterator, seed=seed)

build dense
(?, 384)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 5, 400) (?, 400)


In [16]:
model.model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 5, 384)]     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 5, 50)]      0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 5, 1)]       0                                            
______________________________________________________________________________________________

In [17]:
model._build_naml()[0].summary()

build dense
(?, 384)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 5, 400) (?, 400)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 5, 384)]     0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 5, 50)]      0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, 5, 1)]       0                                            
_

In [18]:
# print(model.run_eval(valid_news_file, valid_behaviors_file))

In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file,valid_news_file, valid_behaviors_file)

0it [00:00, ?it/s]

title shape 51283 


767it [11:24,  3.71it/s]

In [ ]:
%%time
# res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
# print(res_syn)

In [ ]:
# sb.glue("res_syn", res_syn)

## Save the model

In [ ]:
model_path = os.path.join(data_path, "model-naml-lstm")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "naml_ckpt"))

## Output Prediction File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

In [ ]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: Neural News Recommendation with Attentive Multi-View Learning, IJCAI 2019<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/